In [9]:
import pandas as pd
import numpy as np
import flow

df = flow.get_oil_gas_discharge_data()
df.head()

,State,WOR (bbl/bbl),WGR (bbl/Mmcf),Total injected (%),Surface Discharge (%),Evaporation/ Consumption (%)
0,AL,3.3,318.0,0.486,0.514,0.000
1,AK,4.0,0.3,0.999,0.001,0.000
2,AZ,1.3,122.4,1.000,0.000,0.000
3,AR,26.6,9.0,1.000,0.000,0.000
4,CA,15.5,18.5,0.989,0.002,0.009
